# Everest 2 Diagnostics

<br><span style="font-size:150%; font-weight:bold;"><span style="color:gray;">[11/09/16]</span> Optimizing the number of neighbors</span>

Looks like the more neighbors you add, the better you do at the faint end but the worse you do at the bright end. I imagine that this is because adding more stars reduces the amplitude of the regularization parameter, so you end up decreasing the weight of the star's own pixel vectors. For faint stars that's OK, since there's not that much information in them, but for bright stars you end up losing de-trending power. The plot below is the **estats** output comparing a run with 10 neighbors and a run with 50 neighbors. It is clear that 10 neighbors is the way to go.
![10 vs 50 neighbors](images/number_of_neighbors.png "10 vs 50 neighbors")

<br><span style="font-size:150%; font-weight:bold;"><span style="color:gray;">[11/09/16]</span> Neighboring PLD</span>

Plotted below is the comparison between the nPLD(PLD) results (blue) and the Everest 1.0 results (yellow). I've zoomed in on the relative CDPP plot on the right. It looks awesome. Ignore the saturated stars, since I wasn't yet implementing column collapse -- stars brighter than mag 11 are definitely being overfit (issue resolved now!). My current goal is to reproduce this *without* running PLD beforehand to get a factor of 2 speed boost. ![Original nPLD](images/nPLD_original.png "Original nPLD")

<br><span style="font-size:150%; font-weight:bold;"><span style="color:gray;">[11/09/16]</span> Super-detrended cloud</span>

There is a cloud of super-detrended stars just below magnitude 11.5. These stars did not have their columns collapsed since they were just shy of the saturation threshold. Based on the ratio of their max pixel flux to the saturation flux (see list below), I suggest a threshold of -10% when collapsing columns.

![Super-detrended](images/super_detrended.png "Super-detrended")

```0.940860801777, 0.919384317658, 0.944862331189, 0.958934950546, 0.93954328125, 0.946145564303, 0.96732639277, 0.946067420072, 0.944256371287, 0.85614310767, 0.956884969351, 0.931195326593, 0.892291594039, 0.895342764563, 0.940960196554, 0.984913263267, 0.926170642072```

<br><span style="font-size:150%; font-weight:bold;"><span style="color:gray;">[11/10/16]</span> Saturation threshold of -10%</span>

Here's what the nPLD(PLD) model looks like with a saturation threshold of -0.1 (comparing to v1). The super-detrended cloud is gone. I only see two stars below the continuum, so I think this is a good limit.

![Lower saturation threshold](images/saturation_threshold.png "Lower saturation threshold")

<br><span style="font-size:150%; font-weight:bold;"><span style="color:gray;">[11/10/16]</span> Simple nPLD</span>

The downside of nPLD is that the PLD model must be run beforehand, so you're running `everest` twice for each star. We can instead run the model just once -- I call this *simple* nPLD. The difference is that when PLD is run beforehand, we know where the outliers are for each of the neighboring stars, so we mask them when using their pixels as regressors. With *simple* nPLD, the outliers (thruster fires, deep transits, etc.) are present in the regressors. But as it turns out, it makes no difference! In fact, I find slightly **better** de-trending with *simple* nPLD, as shown below.

![Simple](images/simplenPLD.png "Simple nPLD")

The actual light curves look pretty good, too. And comparing the outliers histogram to the nPLD case, this does not introduce extra outliers. Out of curiosity, I also tested what would happen if we did a quick-and-dirty iterative sigma clipping to remove outliers in the regressors. This involves using a guess for the covariance matrix, since I don't want to waste time optimizing the GP. As it turns out, this leads to slightly *worse* de-trending than regular nPLD, so we're not going to do this.

![No Thrusters](images/nothrusters.png "No Thrusters nPLD")

I tested all this on campaign 6.0. The data for the original neighboring PLD scheme is in [stats/NeighborRecursive.tsv](stats/NeighborRecursive.tsv); the data for the new "simple" scheme is in [stats/SimpleNeighborRecursive.tsv](stats/SimpleNeighborRecursive.tsv); and the data for the poor outlier removal simple nPLD is in [stats/NoThrustersRecursive.tsv](stats/NoThrustersRecursive.tsv).

I added a **parent_model** keyword argument to nPLD, which by default will be `None`, meaning nPLD is run on its own (i.e., *simple* nPLD). Setting **parent_model** to "PLD" restores the previous nPLD functionality.

<br><span style="font-size:150%; font-weight:bold;"><span style="color:gray;">[11/10/16]</span> Recursive PLD</span>

While cleaning up the code, I found that I was inadvertently normalizing the pixel weights by the de-trended flux at the current PLD order, rather than by the raw flux. This is what I used to call "recursive" PLD, and I decided against it a while back for some reason. But it turns out that it actually gives you a 1-2% boost in the de-trending power, especially for faint stars. Here I'm comparing recursive nPLD to nPLD (both computed the "simple" way):

![Recursive](images/recursive.png "Recursive nPLD")

Again, these tests were performed on c6.0. The data for the non-recursive simple nPLD is in [stats/SimpleNeighbor.tsv](stats/SimpleNeighbor.tsv).

I've added a **recursive** keyword argument, which from now on will be `True` by default.

<br><span style="font-size:150%; font-weight:bold;"><span style="color:gray;">[11/11/16]</span> Separate Neighbor Regularization</span>

An alternative way to de-trend using neighbors is to separately regularize the neighboring star PLD vectors. Up until now, I was grouping the first order neighbor vectors with the first order target vectors, the second order neighbor vectors with the second order target vectors, and so forth. This stays true to the spirit of orthogonality of the different orders, but a major downside is that it "pollutes" the high de-trending power target vectors with low power neighbor vectors, leading to a suppression of the regularization parameter as the number of neighbors increases. What I did here is to group all of the neighbor vectors (first, second and third order) into a single set and regularize that independently from the target vectors. I call this "separate nPLD". Here I'm comparing separate nPLD to nPLD:

![Separate1](images/separate_neighbors.png "Separate nPLD 1")

It doesn't look good -- in fact, it underperforms nPLD at all magnitudes. The separate nPLD data for c6.0 is in [stats/SeparateNeighbors.tsv](stats/SeparateNeighbors.tsv).

Of course, I cut some corners when I lumped all the orders together. A better way of doing this is to separately regularize each neighboring star order. What I did was to regularize the target's *nth* order signals followed by the neighbors' *nth* order signals for *n = 1, 2, 3*. I optimize the GP only once per order (at the beginning) to save time. I only ran a fraction of the c6.0 stars, but here's how that compares to the separate nPLD approach above:

![Separate2](images/s3n_sn.png "Separate nPLD 2")

It looks better! However, it *still* underperforms our current nPLD method, where we lump all the first order signals together, all the second order signals together, and so forth. Here's the comparison, again for just a subset of the stars:

![Separate3](images/s3n.png "Separate nPLD 3")

The bottom line, I think, is that "separate neighbor regularization" is not worth it. It's slower and actually appears to underperform the traditional method. So that's the last you'll hear about this!

<br><span style="font-size:150%; font-weight:bold;"><span style="color:gray;">[11/11/16]</span> Aggressive GP</span>

I've been thinking of ways to preserve the extreme astrophysical variability seen in some stars. After some testing, I realized that a really simple thing to do is to initialize the de-trending with a much more "aggressive" GP. Instead of initializing the amplitude of the GP at the standard deviation of the data, I now initialize it at 100 times that value. I also raised the upper limit on the amplitude by a factor of 10 to 10,000 times the stddev. It turns out this seems to work really well! The problem used to be that PLD would attempt to fit out the extreme variability, but now that we initialize the fit with a high-power GP, the GP component picks up the slack and prevents PLD over-fitting. Check out EPIC 212793961, which was previously overfit with nPLD (left), but is now well-preserved (right):

![AggressiveGP1](images/212793961_aggressive.png "Aggressive GP")

It's certainly not perfect; EPIC 212300825 is badly overfit with the aggressive nPLD, but looks OK when de-trended with regular nPLD. But it seems like there are more examples where aggressive nPLD does better: 212404864, 212701689, 212760038, 212801119. Here's the results for all of c6.0, compared to regular nPLD:

![AggressiveGP2](images/aggressive.png "Aggressive GP")

In general, it looks like aggressive nPLD actually outperforms nPLD, except for very bright stars. But these are predominantly highly variable red giants, so that's expected! I'm quite happy with this result. The concern, of course, is that this might screw up non-variable stars -- a very strong GP component could prevent PLD from doing its job. It shouldn't lead to overfitting (since we don't include the GP component in the de-trended light curve) -- in fact, I imagine it would lead to underfitting. Based on this figure and on inspecting many of the stars by eye, that doesn't seem to be the case. The data for the aggressive GP run is in [stats/AggressiveGP.tsv](stats/AggressiveGP.tsv). I've added the general model kwargs **gp_factor** to allow the user to tune the "aggressiveness" of the GP; this is the factor by which we multiply the standard deviation of the data when choosing an initial value for the Matern 3/2 kernel amplitude. The default value is 100 (what I used to produce these plots). To restore the original behavior, set this to 1.

<br><span style="font-size:150%; font-weight:bold;"><span style="color:gray;">[11/12/16]</span> Model Names</span>

I've made several changes to the various models, added and removed models, and made a mess of the output files. I'm cleaning everything up today and keeping only two models: ``rPLD`` and ``nPLD``. These are "regular" PLD and "neighboring" PLD. Both include all the changes I mentioned above (an aggressive GP, recursive flux calculations, saturation threshold of -10%, etc.). The default model everywhere in the code will from now on be ``nPLD``.

<br><span style="font-size:150%; font-weight:bold;"><span style="color:gray;">[11/12/16]</span> More ideas</span>

####Fixed Neighbors
One thing to think about is transit pollution with ``nPLD``. Using the neighboring stars' PLD vectors (instead of their SAP flux) is best because it removes astrophysical information. But imagine one star has a transiting planet/eclipsing binary and is in a crowded field, such that its PLD vectors will contain a little bit of transit information. It's possible those transits can get imprinted onto all the stars that use this star's PLD vectors in the de-trending step. **So whenever a planet is found in the ``everest`` light curves, we need to look at all the neighbors used to de-trend and make sure the planet isn't present in any of them.** This is a nuisance. It's possible this isn't going to be a problem, but I will have to do some injection/recovery tests to verify this. For now, what I can do is instead of randomizing neighbors for each star, simply choose (say) 10 stars at the beginning of the de-trending and just use those to de-trend. If we only select stars without any transiting planets, this will never be an issue. (How to know a star doesn't have planets before you de-trend it is a separate problem, but we'll deal with that later). So in this test, I created a "fixed neighbors PLD" model, ``nPLD``, where I use the following stars to de-trend:

212646589,
212500114,
212314869,
212504433,
212817056,
212319518,
212535194,
212812158,
212590172,
212404097,
212598609

Here are the results, compared to the now standard ``nPLD``:

![Fixed](images/fixed_nPLD.png "Fixed nPLD")

Not as good! I don't really know why -- but it seems the targets I chose (bright, well-de-trended, not too variable stars) are not the ideal regressors. Somehow choosing random stars leads to better de-trending (on average). Data stored in [stats/fixed_nPLD.tsv](stats/fixed_nPLD.tsv)

####Three Neighbors
Another thing I tested today was to decrease the number of neighbors from 10 to 3.

![Three](images/three_neighbors.png "Three neighbors")

Significantly worse. Looks like 10 is the sweet spot! Data for this run in [stats/n3PLD.tsv](stats/n3PLD.tsv).

<br><span style="font-size:150%; font-weight:bold;"><span style="color:gray;">[11/13/16]</span> Better Neighbors</span>

In order to prevent transit false positives from showing up when doing ``nPLD`` (see the previous post), from now on we will only use stars that have no nearby bright neighbors, as determined by the MAST search I do to find targets within the postage stamps. This may not find *everything*, and there will likely be some offset between where the MAST positions and where targets actually are on the CCD, but this should solve the problem. I now reject sources with neighbors brighter than Kp - 5, where Kp is the magnitude of the source, within 2 pixels of the edge of their aperture.